# **BUSINESS CASE 3: Recheio Rocommendation System**  


## 🎓 Master’s Program in Data Science & Advanced Analytics 
**Nova IMS** | March 2025   
**Course:** Business Cases with Data Science

## 👥 Team **Group A**  
- **Alice Viegas** | 20240572  
- **Bernardo Faria** | 20240579  
- **Dinis Pinto** | 20240612  
- **Daan van Holten** | 20240681
- **Philippe Dutranoit** | 20240518

## 📊 Project Overview  
This notebook uses the Case3_Recheio_2025 (1).xlsx dataset to build a recommendation system that helps Recheio suggest better products to existing customers.

It addresses two key challenges:<br>
—  How to enrich customer data for more accurate recommendations.<br>
—  How to deliver relevant suggestions across available sales channels. <br>

## 📊 Goal of the notebook

In this notebook we will make smart basket recommendations for the clients for with no transaction information. <br>

**Table of Contents** <br>
* [1. Initial Setup and DataLoading](#setup)
* [2. Best Seller Recommendations](#best)
* [3. Export](#export)

<hr>
<a class="anchor" id="setup">

 ## 1. Initial Setup and Data Loading
 </a>

In [71]:
#Packages
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

In [72]:
clients_no_transactions = pd.read_csv("../Data/clients_without_transactions.csv")
df_transactions = pd.read_csv("../Data/df.csv")
products = pd.read_csv("../Data/products_fixed.csv")

In [73]:
categories = ['']

<hr>
<a class="anchor" id="best">

 ## 2. Best Seller Recommendations
 </a>

### Exploring some populatity rankings for 'Best Selling Products'


In [ ]:
df = df_transactions.copy()
df['Date'] = pd.to_datetime(df['Date'])

# Define time reference
today = df['Date'].max()
df['days_ago'] = (today - df['Date']).dt.days

# Create masks for time windows
mask_7d = df['days_ago'] <= 7
mask_14d = df['days_ago'] <= 14
mask_30d = df['days_ago'] <= 30

# Count appearances per product in each period
sales_7d = df[mask_7d]['ID Product'].value_counts().rename('sales_7d')
sales_14d = df[mask_14d]['ID Product'].value_counts().rename('sales_14d')
sales_30d = df[mask_30d]['ID Product'].value_counts().rename('sales_30d')

# Combine
ranking_df = pd.concat([sales_7d, sales_14d, sales_30d], axis=1).fillna(0)

# Add weekly/monthly hybrid count
ranking_df['hybrid_7_30'] = ranking_df['sales_7d'] * 0.6 + ranking_df['sales_30d'] * 0.4

# Testing some rankings of products
ranking_df['rank_14d'] = ranking_df['sales_14d'].rank(ascending=False, method='min')
ranking_df['rank_hybrid'] = ranking_df['hybrid_7_30'].rank(ascending=False, method='min')

print(ranking_df.sort_values('rank_hybrid').head(10))


            sales_7d  sales_14d  sales_30d  hybrid_7_30  rank_14d  rank_hybrid
ID Product                                                                    
621958         261.0      525.0       1112        601.4       1.0          1.0
879894         177.0      358.0        729        397.8       2.0          2.0
733725         178.0      352.0        715        392.8       3.0          3.0
915237          98.0      193.0        417        225.6       4.0          4.0
659149          89.0      189.0        424        223.0       5.0          5.0
578318          98.0      177.0        386        213.2       6.0          6.0
53425           71.0      166.0        378        193.8       7.0          7.0
890937          64.0      147.0        344        176.0       9.0          8.0
749441          69.0      148.0        310        165.4       8.0          9.0
665738          65.0      131.0        298        158.2      12.0         10.0


Having compared a pure 14-day window with a ***hybrid 7-/30-day approach***, we opted for the latter. 

Prioritizing 7-day sales lets us capture fast-moving items and short-term spikes—driven by promotions, seasonality, or weather—while still incorporating 30-day data to smooth out volatility and spotlight consistently popular products. This blend keeps our recommendations both responsive and reliable, ensuring they respond to emerging trends without sacrificing the stability of proven best-sellers—perfect for a supermarket with varied, rapidly changing demand. 

In [75]:
ranking_df_selection = ranking_df.copy().reset_index()  # 'ID Product' now exists as a column

# Merge with own brand info 
ranking_df_selection = ranking_df_selection.merge(products[['ID Product', 'Own Brand']], on='ID Product', how='left')
ranking_df_selection['Own Brand'] = ranking_df_selection['Own Brand'].fillna(0)

# Apply 20% boost to own brand items in hybrid_7_30 for selection purposes
ranking_df_selection['hybrid_7_30_temp'] = ranking_df_selection['hybrid_7_30'] * (1 + 0.2 * ranking_df_selection['Own Brand'])

# Sort by boosted hybrid score
ranking_df_selection.sort_values('hybrid_7_30_temp', ascending=False, inplace=True)

# Get top 10 product IDs (index should be ID Product)
top10_IDs = ranking_df_selection.head(10)['ID Product'].tolist()


This code adjusts product popularity rankings by giving a 20% boost to own brand items and the DataFrame is sorted by this boosted score. Finally, the top 10 product IDs are selected for recommendation for first time clients.

### Final recommendations

In [76]:
top10 = products[products['ID Product'].isin(top10_IDs)]
top10

,ID Product,Product Description,ID Product Category,Own Brand
101,53425,ARROZ AGULHA MASTERCHEF 5 KG,ARROZ,1
575,370149,CENOURA SC10KG (CAL25/40) RCH,LEGUMES FRESCOS,1
907,578318,OLEO ALIMENTAR MCHEF 10 LT,ÓLEOS,1
974,621958,LEITE MCHEF UHT M/GORDO LT,LEITE UHT REGULAR,1
1083,659149,FARINHA AMANH S/FERMENTO 1KG,FARINHAS,0
1113,665738,VINAGRE AMANHECER DE VINHO BRANCO 1000M,VINAGRES,1
1482,733725,OVO AGRO OVO M 15DUZ IND,OVOS GAIOLA,0
2452,879894,ACUCAR AMANH BCO PAP KG,AÇÚCAR,0
2555,890937,LIXIVIA MCHEF TRADICIONAL 5LT,LIXÍVIAS TRADICIONAIS,1
2897,915237,DET LOICA MCHEF 5LT,DETERGENTE LOIÇA,1


For clients without any transaction history, the top 10 recommended products list will update dynamically over time to reflect the most recent trends in the last 30 days.

<hr>
<a class="anchor" id="export">

## 3. Export
</a>

In [77]:
top10.to_csv('../Data/recommendations_clients_no_transactions.csv', index=False)